In [1]:
import torch
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__Memory Allocated (Device):',torch.cuda.memory_allocated())
    print('__Max Memory Allocated:',torch.cuda.max_memory_allocated()/1e6)
    print('__Memory Reserved:',torch.cuda.memory_reserved()/1e9)
    print('__Currently used device:', torch.cuda.current_device())
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0))#.total_memory/1e6)

__Memory Allocated (Device): 0
__Max Memory Allocated: 0.0
__Memory Reserved: 0.0
__Currently used device: 0
__CUDNN VERSION: 8500
__Number CUDA Devices: 1
__CUDA Device Name: NVIDIA GeForce RTX 2080 SUPER
__CUDA Device Total Memory [GB]: _CudaDeviceProperties(name='NVIDIA GeForce RTX 2080 SUPER', major=7, minor=5, total_memory=7969MB, multi_processor_count=48)


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
!python --version

Python 3.7.13


In [ ]:
!pip3 install requests-html
!pip3 install text-preprocessing

!pip3 install nltk

In [34]:
import os
import csv 
import numpy as np
import pandas as pd
import random
import math
import ast
import re
import regex
import string
#import functools
from functools import reduce
import operator  
import warnings


from collections import OrderedDict
from collections import defaultdict
from datetime import datetime
from datetime import date
from ast import literal_eval

import urllib
import requests
import json
import shutil
import time
import fnmatch
from pathlib import Path

from requests_html import AsyncHTMLSession
from urllib.parse import urlparse
from requests_html import HTMLSession

#import PyPaperBot
import nltk
#import fitz
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer



#from scidownl import scihub_download
import pickle
from text_preprocessing import preprocess_text
from text_preprocessing import *
# from sentence_transformers import SentenceTransformer
# from sentence_transformers import models
# from sentence_transformers import util

#import torch
#from transformers import AutoTokenizer, AutoModel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
from functools import reduce
import operator  
import warnings

from itertools import groupby

#from requests_html import HTMLSession

# import nest_asyncio
# nest_asyncio.apply()

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



In [35]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/psych256lab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:

vocab = set(w.lower() for w in nltk.corpus.words.words())
lemmatizer = WordNetLemmatizer()

In [37]:
# for reproducibility
np.random.seed(1)

In [100]:
filterd_articlePolicyList2 = pd.read_csv('filterd_articlePolicyList.csv', encoding='utf-8')

In [101]:
filterd_articlePolicyList2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         2301 non-null   object 
 1   title                       2301 non-null   object 
 2   policyList                  2301 non-null   object 
 3   policyTitles                2301 non-null   object 
 4   USApolicyCitation           2301 non-null   int64  
 5   successful_policyList       2301 non-null   object 
 6   successful_policyTitles     2301 non-null   object 
 7   articlePdf                  2301 non-null   object 
 8   total_successful_policy     2301 non-null   int64  
 9   successfulPolicyPercentage  2301 non-null   float64
dtypes: float64(1), int64(2), object(7)
memory usage: 179.9+ KB


In [102]:
filterd_articlePolicyList2.set_index('url', inplace =True)

In [103]:
filterd_articlePolicyList2.at['https://doi.org/10.1001/archderm.139.3.318','articlePdf']

'5_the frequency of common skin conditions in preschoolaged children in australia.pdf'

In [104]:
filterd_articlePolicyList2.policyTitles = filterd_articlePolicyList2.policyTitles.apply(literal_eval)
filterd_articlePolicyList2.policyList =  filterd_articlePolicyList2.policyList.apply(literal_eval)                
filterd_articlePolicyList2.successful_policyList = filterd_articlePolicyList2.successful_policyList.apply(literal_eval)
filterd_articlePolicyList2.successful_policyTitles = filterd_articlePolicyList2.successful_policyTitles.apply(literal_eval)
#filterd_articlePolicyList2.articlePdf = filterd_articlePolicyList2.articlePdf.apply(literal_eval)

In [105]:
new_similarity_column = 'tfidf_sim'
filterd_articlePolicyList2.insert(len(filterd_articlePolicyList2.columns), new_similarity_column, '[]')
filterd_articlePolicyList2[new_similarity_column] = filterd_articlePolicyList2[new_similarity_column].apply(literal_eval)

In [106]:
filterd_articlePolicyList2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2301 entries, https://doi.org/10.1001/archderm.138.12.1584 to https://doi.org/10.9778/cmajo.20190026
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   title                       2301 non-null   object 
 1   policyList                  2301 non-null   object 
 2   policyTitles                2301 non-null   object 
 3   USApolicyCitation           2301 non-null   int64  
 4   successful_policyList       2301 non-null   object 
 5   successful_policyTitles     2301 non-null   object 
 6   articlePdf                  2301 non-null   object 
 7   total_successful_policy     2301 non-null   int64  
 8   successfulPolicyPercentage  2301 non-null   float64
 9   tfidf_sim                   2301 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 262.3+ KB


In [107]:
# #Load sentences & embeddings from disc
with open('policy_sentences.pkl', "rb") as fIn:
    policy_sentences = pickle.load(fIn)

In [108]:
# #Load sentences & embeddings from disc
with open('article_abstract_sentences.pkl', "rb") as fIn:
    article_sentences = pickle.load(fIn)
#filterd_articlePolicyList2.info()
   

In [ ]:
article_sentences['https://doi.org/10.9778/cmajo.20190026']  


In [110]:
vectorizer = TfidfVectorizer()

In [111]:
def calculateArticlePolicySimilarity(index: str, article_doc: str, policy_doc: str)->None:

    tfidf_matrix = vectorizer.fit_transform([article_doc, policy_doc])
    similarity_score = float("{:0.3f}".format(cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]))
    filterd_articlePolicyList2.at[index,'tfidf_sim'] += [similarity_score]
    
    return None

In [112]:
# #original 
# def getPreprocessedDocument(index: str, policy_index: int, policy_url: str, file_type: str)-> str:

#     url = index
#     type_sentences = article_sentences
#     doc =''

#     if file_type=='policy':
#         url = policy_url
#         type_sentences = policy_sentences
#     if (url in type_sentences):
#         sentence_list = type_sentences[url]
#         doc = [nltk.word_tokenize(sentence.lower()) for sentence in sentence_list if sentence != ''] #doc.split()
#         doc = reduce(operator.iconcat, doc, []) #flatten list of lists of tokens
#         #print("split: ", doc)
#         doc = ' '.join([lemmatizer.lemmatize(word) for word in doc if not word in set(stop_words)])
                       
#     return doc

In [113]:
remove_token = list(string.ascii_lowercase[1:26]) +['al','mo','cox']

In [114]:
def sentenceProcessing1(sentence:str)->list:
    token_list = nltk.word_tokenize(sentence.lower())
    filtered_token_list = [token for token in token_list if token not in remove_token if token not in stop_words if token in vocab ] #
    filtered_token_list1 = [i[0] for i in groupby(filtered_token_list)] #removes consecutive duplicate words
    # filtered_token_list2 = [token for token in filtered_token_list1 if token not in stop_words]
    #if len(filtered_token_list2)!=0:
    return filtered_token_list1
    #else:
     #   return list()

In [122]:
def getPreprocessedDocument1(index: str, policy_index: int, policy_url: str, file_type: str)-> str:

    url = index
    type_sentences = article_sentences
    title = filterd_articlePolicyList2.at[index,'title'] # for title-title 
    doc =''

    if file_type=='policy':
        url = policy_url
        type_sentences = policy_sentences
        policy_pdf_title = filterd_articlePolicyList2.at[index,'successful_policyTitles'][policy_index] # for title-title
        title = policy_pdf_title # # for title-title 
    if (url in type_sentences):
        # sentence_list = type_sentences[url]
        # if (len(sentence_list)!=0):
        if (len(title)!=0):
            # tokenized_sentence_list = [sentenceProcessing1(sentence) for sentence in sentence_list if sentence != ''] #doc.split()
            doc = sentenceProcessing1(title) #for sentence in title if sentence != ''] #doc.split()
            # # preprocessed_article= ' '.join([TreebankWordDetokenizer().detokenize(tokenized_sentence) for tokenized_sentence in tokenized_sentence_list if len(tokenized_sentence)!=0])
            # # doc = [nltk.word_tokenize(sentence.lower()) for sentence in sentence_list if sentence != ''] #doc.split()
            # doc = reduce(operator.iconcat, tokenized_sentence_list, []) #flatten list of lists of tokens
            # #print("split: ", doc)
            doc = ' '.join([lemmatizer.lemmatize(word) for word in doc if not word in set(stop_words)])

    return doc

In [123]:
# def textProcessing(index: str):  # article is a list of sentences
#     if index in article_sentences:
#         article_sentence_list = article_sentences[index]
#         if (len(article_sentence_list)!=0):
#             tokenized_sentence_list = [sentenceProcessing(sentence) for sentence in article_sentence_list if sentence != ''] #doc.split()
#             preprocessed_article= ' '.join([TreebankWordDetokenizer().detokenize(tokenized_sentence) for tokenized_sentence in tokenized_sentence_list if len(tokenized_sentence)!=0])

#             lexical_analysis(index, preprocessed_article)
#     ### call function for measurements
    
#     return None

In [124]:
#for index in df:
def getTfIdfSimilarity(index:str)->None:
    # For Articles
    article_url = index #filterd_articlePolicyList2.at[index,'url']
    preprocessed_article = getPreprocessedDocument1(index, policy_index = -1, policy_url='', file_type = 'article')
    if (len(preprocessed_article)):
        policy_urls= filterd_articlePolicyList2.at[index,'successful_policyList']
        preprocessed_policy_list =  [getPreprocessedDocument1(index, policy_index, policy_url, file_type = 'policy') for policy_index, policy_url in enumerate(policy_urls)]
                        # List of embedding dictionaries
        output = [calculateArticlePolicySimilarity(index, preprocessed_article,preprocessed_policy)  for preprocessed_policy in preprocessed_policy_list if len(preprocessed_policy_list)!=0 and len(preprocessed_policy)!=0]      
    #         filterd_articlePolicyList2.at[index,'final_policy_pdfs'] += [policy_pdf_title]           

    return None

In [125]:
# article_sentences['https://doi.org/10.9778/cmajo.20190026'] 

In [126]:
article_list = list(filterd_articlePolicyList2.index)

In [127]:
# getTfIdfSimilarity('https://doi.org/10.9778/cmajo.20190026')
# filterd_articlePolicyList2.at['https://doi.org/10.9778/cmajo.20190026','tfidf_sim']

In [131]:
filterd_articlePolicyList2#.info()#['tfidf_sim'].tail(500)

,title,policyList,policyTitles,USApolicyCitation,successful_policyList,successful_policyTitles,articlePdf,total_successful_policy,successfulPolicyPercentage,tfidf_sim
url,,,,,,,,,,
https://doi.org/10.1001/archderm.138.12.1584,acne vulgaris,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Approaches to limit systemic antibiotic use i...,1,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[3_0_approaches to limit systemic antibiotic u...,3_acne vulgaris.pdf,1,100.00,[0.21]
https://doi.org/10.1001/archderm.139.3.318,the frequency of common skin conditions in pre...,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Topical antifungals for seborrhoeic dermatiti...,3,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[5_1_interventions for infantile seborrhoeic d...,5_the frequency of common skin conditions in p...,3,100.00,"[0.0, 0.0]"
https://doi.org/10.1001/archgenpsychiatry.2009.173,brief personalitytargeted coping skills interv...,[https://purl.fdlp.gov/GPO/gpo75951],[Facing addiction in America : the Surgeon Gen...,1,[https://purl.fdlp.gov/GPO/gpo75951],[16_0_facing addiction in america the surgeon...,16_brief personalitytargeted coping skills int...,1,100.00,[0.0]
https://doi.org/10.1001/archgenpsychiatry.2009.186,childhood adversities and adult psychiatric di...,[https://www.aclu.org/legal-document/rilr-v-jo...,[RILR v. Johnson - Amended Motion for Prelimin...,6,[https://hhs.texas.gov/sites/default/files/doc...,[17_3_the mommies toolkit improving outcomes f...,17_childhood adversities and adult psychiatric...,4,66.67,"[0.0, 0.0, 0.0, 0.0]"
https://doi.org/10.1001/archgenpsychiatry.2010.168,antidepressant monotherapy vs sequential pharm...,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Approaches for discontinuation versus continu...,5,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[20_1_canadian network for mood and anxiety tr...,20_antidepressant monotherapy vs sequential ph...,4,80.00,"[0.0, 0.0, 0.0, 0.093]"
...,...,...,...,...,...,...,...,...,...,...
https://doi.org/10.9745/ghsp-d-18-00275,mlearning in the democratic republic of the co...,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Health workers’ perceptions and experiences o...,1,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[52264_0_health workers perceptions and experi...,52264_mlearning in the democratic republic of ...,1,100.00,[0.0]
https://doi.org/10.9745/ghsp-d-19-00195,effectiveness of an electronic partogram a mix...,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[The injustice of unfit clinical practice guid...,1,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[52266_0_the injustice of unfit clinical pract...,52266_effectiveness of an electronic partogram...,1,100.00,[0.0]
https://doi.org/10.9758/cpn.2013.11.3.107,the neuroimmunology of schizophrenia,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Acetylsalicylic acid (aspirin) for schizophre...,1,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[52270_0_acetylsalicylic acid aspirin for schi...,52270_the neuroimmunology of schizophrenia.pdf,1,100.00,[0.0]


In [129]:
out = [getTfIdfSimilarity(article) for article in article_list] # if article_list.index(article)>931]

In [132]:
filterd_articlePolicyList2.to_csv("ttl_tfidf_sim.csv",encoding = 'utf-8',index=False)

In [40]:
#!pip3 install GPUtil
#import torch
import gc
from GPUtil import showUtilization as gpu_usage
from numba import cuda
print(gpu_usage())
gc.collect()
torch.cuda.empty_cache()
gpu_usage()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=5633f9d54bf5bd10e33664b16b7f03fc6ef9aaab1c76873d92bc747462bb15b1
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
None
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [134]:
filterd_articlePolicyList2

,url,title,policyList,policyTitles,USApolicyCitation,successful_policyList,successful_policyTitles,articlePdf,total_successful_policy,successfulPolicyPercentage,tfidf_sim
0,https://doi.org/10.1001/archderm.138.12.1584,acne vulgaris,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Approaches to limit systemic antibiotic use i...,1,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[3_0_approaches to limit systemic antibiotic u...,3_acne vulgaris.pdf,1,100.00,[0.21]
1,https://doi.org/10.1001/archderm.139.3.318,the frequency of common skin conditions in pre...,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Topical antifungals for seborrhoeic dermatiti...,3,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[5_1_interventions for infantile seborrhoeic d...,5_the frequency of common skin conditions in p...,3,100.00,"[0.0, 0.0]"
2,https://doi.org/10.1001/archgenpsychiatry.2009...,brief personalitytargeted coping skills interv...,[https://purl.fdlp.gov/GPO/gpo75951],[Facing addiction in America : the Surgeon Gen...,1,[https://purl.fdlp.gov/GPO/gpo75951],[16_0_facing addiction in america the surgeon...,16_brief personalitytargeted coping skills int...,1,100.00,[0.0]
3,https://doi.org/10.1001/archgenpsychiatry.2009...,childhood adversities and adult psychiatric di...,[https://www.aclu.org/legal-document/rilr-v-jo...,[RILR v. Johnson - Amended Motion for Prelimin...,6,[https://hhs.texas.gov/sites/default/files/doc...,[17_3_the mommies toolkit improving outcomes f...,17_childhood adversities and adult psychiatric...,4,66.67,"[0.0, 0.0, 0.0, 0.0]"
4,https://doi.org/10.1001/archgenpsychiatry.2010...,antidepressant monotherapy vs sequential pharm...,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Approaches for discontinuation versus continu...,5,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[20_1_canadian network for mood and anxiety tr...,20_antidepressant monotherapy vs sequential ph...,4,80.00,"[0.0, 0.0, 0.0, 0.093]"
...,...,...,...,...,...,...,...,...,...,...,...
2296,https://doi.org/10.9745/ghsp-d-18-00275,mlearning in the democratic republic of the co...,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Health workers’ perceptions and experiences o...,1,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[52264_0_health workers perceptions and experi...,52264_mlearning in the democratic republic of ...,1,100.00,[0.0]
2297,https://doi.org/10.9745/ghsp-d-19-00195,effectiveness of an electronic partogram a mix...,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[The injustice of unfit clinical practice guid...,1,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[52266_0_the injustice of unfit clinical pract...,52266_effectiveness of an electronic partogram...,1,100.00,[0.0]
2298,https://doi.org/10.9758/cpn.2013.11.3.107,the neuroimmunology of schizophrenia,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Acetylsalicylic acid (aspirin) for schizophre...,1,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[52270_0_acetylsalicylic acid aspirin for schi...,52270_the neuroimmunology of schizophrenia.pdf,1,100.00,[0.0]
2299,https://doi.org/10.9758/cpn.2013.11.3.118,transcranial direct current stimulation in sch...,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Evidence-Based Guidelines and Secondary Meta-...,1,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[52271_0_evidencebased guidelines and secondar...,52271_transcranial direct current stimulation ...,1,100.00,[0.407]


In [133]:
filterd_articlePolicyList2.reset_index('url',inplace =True)

In [135]:
filterd_articlePolicyList2.to_csv('ttl_tfidf_sim.csv', encoding ='utf-8', index = False)

In [ ]:
def textProcessing(res:list):
    # Stopword removal, converting uppercase into lower case, and lemmatization
    data_without_stopwords = []
    
    for i in range(0, len(res)):
        doc = [nltk.word_tokenize(sentence).lower() for sentence in sentList] #doc.split()
        functools.reduce(operator.iconcat, doc, [])
        print("split: ",doc)
        doc = ' '.join([lemmatizer.lemmatize(word) for word in doc if not word in set(stopwords)])
        # print("join: ",doc)
        data_without_stopwords += [doc]
    return data_without_stopwords

response = ["Good muffins cost $3.88 in New York.  Please buy me","Nick likes to play football, however he is s not too fond  of t'tennis.","The striped bats play were hanging on their feet and ate best fishes"]
doc1=textProcessing(response)
print(doc1)
#make a single list of words for each sentence (flat)
#do the above for article and policy and pass below

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([doc1[2], doc1[1]])
similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
similarity_score